## SRJ - FeS

In [11]:
## Usage of basis set
import basis_set_exchange as bse
#print(bse.get_all_basis_names())

basis = bse.get_basis('ANO-RCC-MB', fmt='nwchem')
#print(basis)

import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize

#qml.data.load("qchem", molname = "FeS", basis = basis)

n_basis = qml.qchem.mol_basis_data('ANO-RCC-MB', ['Fe', 'S'], load_data=basis)
print(n_basis)


([19, 9], (((0, 0, 0), [4316265.0, 646342.4, 147089.7, 41661.52, 13590.77, 4905.75, 1912.746, 792.6043, 344.8065, 155.8999, 72.23091, 32.72506, 15.66762, 7.503483, 3.312223, 1.558471, 0.683914, 0.146757, 0.070583, 0.031449, 0.01258], [0.00015003, 0.00043597, 0.00120365, 0.00312635, 0.00814591, 0.02133892, 0.05470838, 0.12845394, 0.25203824, 0.35484986, 0.27043078, 0.06476086, -0.00110466, 0.00184555, -0.000856, 0.00037119, -0.00014687, 6.097e-05, -5.789e-05, 2.77e-05, -7.22e-06]), ((0, 0, 0), [4316265.0, 646342.4, 147089.7, 41661.52, 13590.77, 4905.75, 1912.746, 792.6043, 344.8065, 155.8999, 72.23091, 32.72506, 15.66762, 7.503483, 3.312223, 1.558471, 0.683914, 0.146757, 0.070583, 0.031449, 0.01258], [-4.622e-05, -0.00013445, -0.00037184, -0.00096889, -0.00253948, -0.00673001, -0.0176816, -0.0437541, -0.09601111, -0.16998599, -0.18456376, 0.058263, 0.52163758, 0.49331062, 0.0863267, -0.00285017, 0.00165569, -0.00049176, 0.00047608, -0.0002282, 6.297e-05]), ((0, 0, 0), [4316265.0, 646342

In [10]:
n_basis

([19, 9],
 (((0, 0, 0),
   [4316265.0,
    646342.4,
    147089.7,
    41661.52,
    13590.77,
    4905.75,
    1912.746,
    792.6043,
    344.8065,
    155.8999,
    72.23091,
    32.72506,
    15.66762,
    7.503483,
    3.312223,
    1.558471,
    0.683914,
    0.146757,
    0.070583,
    0.031449,
    0.01258],
   [0.00015003,
    0.00043597,
    0.00120365,
    0.00312635,
    0.00814591,
    0.02133892,
    0.05470838,
    0.12845394,
    0.25203824,
    0.35484986,
    0.27043078,
    0.06476086,
    -0.00110466,
    0.00184555,
    -0.000856,
    0.00037119,
    -0.00014687,
    6.097e-05,
    -5.789e-05,
    2.77e-05,
    -7.22e-06]),
  ((0, 0, 0),
   [4316265.0,
    646342.4,
    147089.7,
    41661.52,
    13590.77,
    4905.75,
    1912.746,
    792.6043,
    344.8065,
    155.8999,
    72.23091,
    32.72506,
    15.66762,
    7.503483,
    3.312223,
    1.558471,
    0.683914,
    0.146757,
    0.070583,
    0.031449,
    0.01258],
   [-4.622e-05,
    -0.00013445,
    -0

In [13]:
#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import itertools
import time
import re
import scipy
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []
old_grad = []
excitationlist = []
generatingfns = []
gs_energy = []
theta = np.pi/4
operator_check = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity



bond_length = 1.826
symbols = ["Fe","S"]
electrons = 4
orbitals = 12
r_bohr = bond_length *1.8897259886  
coordinates = np.array([[0.0,0.0, 0*r_bohr], [0.0, 0.0, 1*r_bohr]])
molecule = qml.qchem.Molecule(symbols, coordinates, basis_name='ano-rcc-mb')


H, qubits = qml.qchem.molecular_hamiltonian(molecule)
#hf_state = qchem.hf_state(electrons, qubits)

def adaptvqe(adapt_it = 4, e_th=1e-12):
    ash_excitation = []
    energies = []
    excitations= []
    #Preparation of the Hartree-Fock state in the form of 2^8 states
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def hf_stateprep(wires):
        target_state = np.zeros(2**qubits)
        target_state[3840] = 1.0 # Every molecule change, you need to change this index #3840
        qml.StatePrep(target_state, wires=range(qubits))
        return qml.state()

    hf_state = hf_stateprep(wires=qubits)
    print('HF state is', hf_state)

    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, electrons, qubits, H):
        #print('Updated hf_state is', hf_state)  
        qml.StatePrep(hf_state, wires=range(qubits))
        return qml.expval(H)  
    
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the ash_excitationis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)
    #Applyign the givens rotation to the HF state
    #For generating the new state, they are adding the excitation in the exponent of the rotation, thats 
    #why we need to use qml.FermionicSE or qml.FermionicDE
    @qml.qnode(dev)
    def new_state(hf_state, ash_excitation, qubits):
        qml.StatePrep(hf_state, wires=range(qubits))
        for i in range(len(ash_excitation)):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=theta, wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=theta, wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
     
    #Measurement of new state
    dev_meas = qml.device("lightning.qubit", wires=qubits, shots=1000)
    @qml.qnode(dev_meas)
    def measure(ostate):
        qml.StatePrep(ostate, wires=range(qubits))
        return qml.counts()
    
    @qml.qnode(dev)
    def ind_state(ash_excitation):
        qml.StatePrep(hf_state, wires=range(qubits))
        print('Individual excitation is', ash_excitation)
        if len(ash_excitation) == 4 :
            qml.DoubleExcitation(theta, wires=ash_excitation)
        elif len(ash_excitation) == 2:
            qml.SingleExcitation(theta, wires=ash_excitation)
        return qml.state()


    print('HF state is', circuit(hf_state, electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(electrons, orbitals)

            #print('D-Generating functions are', super2)
    #print('Total len of Generating functions is', len(generatingfns))
    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    states = [hf_state]
    print('The length of operator pool', len(operator_pool))


    for j in range(1, adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
        
        
        for i in operator_pool:
            if i not in operator_check:  # Check if the operator is already checked
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
                w = qml.fermi.jordan_wigner(i)  #JW transformation
                if np.array_equal(k, hf_state): # If the current state is the HF state
                    #print('Print, if this is activated - HF state')
                    current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
                else:
                    current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated
                #print(f'The expectation value of {i} is', current_value)

                if current_value > max_value:
                    max_value = current_value
                    max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value
        operator_check.append(max_operator)

        # Convert operator to excitations and append to ash_excitation
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations1 = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations1)
        ash_excitation.append(excitations1)
        print('ash_excitation is', ash_excitation)
        print('The length of ash_excitation before generating matrix is', len(ash_excitation))

        
        #Now apply the givens rotation for it. 
        ostate = new_state(hf_state, ash_excitation, qubits)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation, qubits))
        #Measuring the state
        counts = measure(ostate)
        print('State count after Generating function is', counts)
        states.append(ostate)
        if j >= 2:
            states.append(ind_state(ash_excitation[-1]))  # Append the individual excitation state
        else:
            print('Skipping individual excitation state for j < 2')
        print('After adding new state, the length of states is', len(states))
        M = np.zeros((len(states), len(states)), dtype=complex)  # Initialize H with zeros
        S = np.zeros((len(states), len(states)), dtype=complex)  # Initialize S matrix with zeros
        Ham_matrix = qml.matrix(H)
        print('Ham matrix is', Ham_matrix)
        # Now we need to fill the H and S matrices
        for i in range(len(states)):
            for j in range(len(states)):
                print('The value of i and j is', i, j)
                S[i,j] = (states[i].conj().T) @ states[j] #<psi_i|psi_j>
                M[i,j] = (states[i].conj().T) @ Ham_matrix @ states[j]
                if i != j:
                    M[j,i] = M[i,j].conj()
                    S[j,i] = S[i,j].conj()
        n = S.shape[0]
        epsilon = 1e-10
        S_reg = S + epsilon * np.eye(n)
        print(f'S matrix with {i} and {j} is', S)
        print('Hamiltonian Matrix with', i, j, 'is', M)
        eig,evec=scipy.linalg.eigh(M,S_reg)
        print('Eigenvalues are', eig)
        gs_energy.append(eig[0])
        print('Ground state energy is', gs_energy)    
    return ash_excitation, states,eig,gs_energy, Ham_matrix


ValueError: Currently, the only supported basis sets are 'sto-3g', '6-31g', '6-311g' and 'cc-pvdz'. Please consider using `load_data=True` to download the basis set from an external library that can be installed with: pip install basis-set-exchange.